In [ ]:
import os, sys
sys.path.append('/root/catkin_ws/src/primitives/')
import pickle
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import axes3d
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import trimesh
import networkx

from open3d import JVisualizer

In [ ]:
%matplotlib notebook

In [ ]:
os.system('export CODE_BASE=/root/')

In [ ]:
import os, sys
import time
import argparse
import numpy as np
from multiprocessing import Process, Pipe, Queue
import pickle
import rospy
import copy
import signal
import open3d
from IPython import embed

from airobot import Robot
from airobot.utils import pb_util
from airobot.sensor.camera.rgbdcam_pybullet import RGBDCameraPybullet
from airobot.utils import common
import pybullet as p

from helper import util
from macro_actions import ClosedLoopMacroActions, YumiGelslimPybulet
# from closed_loop_eval import SingleArmPrimitives, DualArmPrimitives

from yacs.config import CfgNode as CN
from closed_loop_experiments import get_cfg_defaults

In [ ]:
with open('/root/catkin_ws/src/primitives/data/pull/face_ind_large_0/metadata.pkl', 'rb') as mf:
    metadata = pickle.load(mf)
# with open('/root/catkin_ws/src/primitives/data/grasp/face_ind_test_0/metadata.pkl', 'rb') as mf:
#     metadata = pickle.load(mf)


print('Metadata keys: ')
print(metadata.keys())
print(metadata['cfg'])
cfg = metadata['cfg']
step_repeat = metadata['step_repeat']
dynamics_info = metadata['dynamics']
mesh_file = metadata['mesh_file']

palm_mesh_file = '/root/catkin_ws/src/config/descriptions/meshes/mpalm/mpalms_all_coarse.stl'
# table_mesh_file = '../catkin_ws/src/config/descriptions/meshes/table/coarse/table_base.stl'
table_mesh_file = '/root/catkin_ws/src/config/descriptions/meshes/table/table_top.stl'

In [ ]:
with open ('/root/catkin_ws/src/primitives/data/pull/face_ind_large_0/3.pkl', 'rb') as f:
    data = pickle.load(f)
# with open ('/root/catkin_ws/src/primitives/data/grasp/face_ind_test_0/2.pkl', 'rb') as f:
#     data = pickle.load(f)

In [ ]:
for key in data.keys():
    if key != 'obs':
        print(key)
        print(data[key])

# Visualize contact on object

In [ ]:
obj_mesh = trimesh.load_mesh(mesh_file)
r_palm_mesh = trimesh.load_mesh(palm_mesh_file)
l_palm_mesh = trimesh.load_mesh(palm_mesh_file)

In [ ]:
obj_pos_world = data['start'][:3]
obj_ori_world = data['start'][3:]
obj_ori_mat = common.quat2rot(obj_ori_world)
h_trans = np.zeros((4, 4))
h_trans[:3, :3] = obj_ori_mat
h_trans[:-1, -1] = obj_pos_world
h_trans[-1, -1] = 1

obj_mesh.apply_transform(h_trans)

In [ ]:
tip_contact_r = util.list2pose_stamped(data['contact_world_frame']['right'])
tip_contact_l = util.list2pose_stamped(data['contact_world_frame']['left'])

wrist_contact_r = util.convert_reference_frame(
    pose_source=util.list2pose_stamped(cfg.TIP_TO_WRIST_TF),
    pose_frame_target=util.unit_pose(),
    pose_frame_source=tip_contact_r)

wrist_contact_l = util.convert_reference_frame(
    pose_source=util.list2pose_stamped(cfg.TIP_TO_WRIST_TF),
    pose_frame_target=util.unit_pose(),
    pose_frame_source=tip_contact_l)

wrist_contact_r_list = util.pose_stamped2list(wrist_contact_r)
wrist_contact_l_list = util.pose_stamped2list(wrist_contact_l)

In [ ]:
tip_contact_r = util.list2pose_stamped(data['contact_world_frame'])

wrist_contact_r = util.convert_reference_frame(
    pose_source=util.list2pose_stamped(cfg.TIP_TO_WRIST_TF),
    pose_frame_target=util.unit_pose(),
    pose_frame_source=tip_contact_r)

# wrist_contact_l = util.convert_reference_frame(
#     pose_source=util.list2pose_stamped(cfg.TIP_TO_WRIST_TF),
#     pose_frame_target=util.unit_pose(),
#     pose_frame_source=tip_contact_l)

wrist_contact_r_list = util.pose_stamped2list(wrist_contact_r)
# wrist_contact_l_list = util.pose_stamped2list(wrist_contact_l)

In [ ]:
palm_pos_world_r = wrist_contact_r_list[:3]
palm_ori_world_r = wrist_contact_r_list[3:]
palm_ori_mat = common.quat2rot(palm_ori_world_r)
h_trans = np.zeros((4, 4))
h_trans[:3, :3] = palm_ori_mat
h_trans[:-1, -1] = palm_pos_world_r
h_trans[-1, -1] = 1

r_palm_mesh.apply_transform(h_trans)

In [ ]:
palm_pos_world_l = wrist_contact_l_list[:3]
palm_ori_world_l = wrist_contact_l_list[3:]
palm_ori_mat = common.quat2rot(palm_ori_world_l)
h_trans = np.zeros((4, 4))
h_trans[:3, :3] = palm_ori_mat
h_trans[:-1, -1] = palm_pos_world_l
h_trans[-1, -1] = 1

l_palm_mesh.apply_transform(h_trans)

In [ ]:
table_mesh = trimesh.load_mesh(table_mesh_file)

In [ ]:
scene = trimesh.Scene([obj_mesh, r_palm_mesh, table_mesh])

In [ ]:
scene.show(viewer='gl')

# Visualize point cloud

In [ ]:
# pcd = data['obs']['pcd_full']
pcd_pts = data['obs']['pcd_pts']
obj_pcd_pts_cat = np.concatenate(pcd_pts, axis=0)

In [ ]:
obj_pcd_pts_cat.shape

In [ ]:
fig = go.Figure(data=go.Scatter3d(
    x=obj_pcd_pts_cat[:, 0],
    y=obj_pcd_pts_cat[:, 1],
    z=obj_pcd_pts_cat[:, 2]))

fig.update_xaxes(range=[-0.25, 0.7])
fig.update_yaxes(range=[-0.75, 0.75])

fig.show()

fig.update_xaxes(range=[-0.25, 0.7])
fig.update_yaxes(range=[-0.75, 0.75])

In [ ]:
# visualizer = JVisualizer()
# visualizer.add_geometry([pcd])
# visualizer.show()

# open3d.visualization.draw_geometries([pcd])

# Visualize image observations

In [ ]:
im_fig = plt.figure()

ax1 = im_fig.add_subplot(311)
ax1.imshow(data['obs']['rgb'][0])

ax2 = im_fig.add_subplot(312)
ax2.imshow(data['obs']['rgb'][1])

ax3 = im_fig.add_subplot(313)
ax3.imshow(data['obs']['rgb'][2])

plt.show()

# Visualize keypoints

In [ ]:
kp_fig_0 = go.Figure(data=go.Scatter3d(
        x=data['keypoints_start'][:, 0],
        y=data['keypoints_start'][:, 1],
        z=data['keypoints_start'][:, 2],
        mode='markers'))

kp_fig_0.show()

In [ ]:
kp_fig = make_subplots(rows=1, cols=2,  specs=[[{"type": "scene"}, {"type": "scene"}]])

kp_fig.add_trace(go.Scatter3d(
        x=data['keypoints_start'][:, 0],
        y=data['keypoints_start'][:, 1],
        z=data['keypoints_start'][:, 2],
        mode='markers'), 
    row=1, col=1)

kp_fig.add_trace(go.Scatter3d(
        x=data['keypoints_goal'][:, 0],
        y=data['keypoints_goal'][:, 1],
        z=data['keypoints_goal'][:, 2],
        mode='markers'),
    row=1, col=2)

kp_fig.show()

In [ ]:
# try visulizing the meshes with the table for showing the poses?



# Visualize trajectory in PyBullet

In [ ]:
rospy.init_node("test")

In [ ]:
yumi = Robot('yumi_palms', arm_cfg={'render': True, 'self_collision': False, 'rt_simulation': True})
yumi.arm.go_home()
yumi.arm.set_jpos(cfg.RIGHT_INIT + cfg.LEFT_INIT)

In [ ]:
gel_id = 12

p.changeDynamics(
    yumi.arm.robot_id,
    gel_id,
    restitution=dynamics_info['restitution'],
    contactStiffness=dynamics_info['contactStiffness'],
    contactDamping=dynamics_info['contactDamping'],
    rollingFriction=dynamics_info['rollingFriction']
)

In [ ]:
yumi_gs = YumiGelslimPybulet(
    yumi,
    cfg)

In [ ]:
box_id = pb_util.load_urdf(
    '/root/catkin_ws/src/config/descriptions/urdf/realsense_box.urdf',
    cfg.OBJECT_POSE_3[0:3],
    cfg.OBJECT_POSE_3[3:]
)

In [ ]:
config_pkg_path = '/root/catkin_ws/src/config/'

action_planner = ClosedLoopMacroActions(
    cfg,
    yumi_gs,
    box_id,
    pb_util.PB_CLIENT,
    config_pkg_path,
    object_mesh_file=mesh_file,
    replan=True
)

planner_args = data['planner_args']
primitive_name = planner_args['primitive_name']
object_start_pose_list = data['start']
object_goal_pose_list = data['goal']

pb_util.reset_body(
    body_id=box_id, 
    base_pos=object_start_pose_list[:3],
    base_quat=object_start_pose_list[3:])

In [ ]:
result = action_planner.execute(primitive_name=primitive_name, execute_args=planner_args)

In [ ]:
new_args = {}
new_args['object_pose1_world'] = list2pose_stamped(object_start_pose_list)
new_args['object_pose2_world'] = list2pose_stamped(object_goal_pose_list)
new_args['primitive_name'] = 'pull'
new_args['palm_pose_r_object'] = list2pose_stamped(data['contact_obj_frame'])
new_args['palm_pose_l_object'] = list2pose_stamped(cfg.PALM_LEFT)
new_args['object'] = None
new_args['init'] = True
new_args['N'] = 32
new_args['table_face'] = 0

In [ ]:
from planning import pulling_planning

manipulated_object = new_args['object']
object_pose1_world = new_args['object_pose1_world']
object_pose2_world = new_args['object_pose2_world']
palm_pose_l_object = new_args['palm_pose_l_object']
palm_pose_r_object = new_args['palm_pose_r_object']
table_face = new_args['table_face']
active_arm = 'right'
N = new_args['N']

plan = pulling_planning(
    object=manipulated_object,
    object_pose1_world=object_pose1_world,
    object_pose2_world=object_pose2_world,
    palm_pose_l_object=palm_pose_l_object,
    palm_pose_r_object=palm_pose_r_object,
    arm=active_arm[0],
    N=N)